# Lesson 5 - Assignment

In this assignment, you will implement a Support Vector Machine Classifier  from scratch and compare the results to existing sklearn algorithm. 

In [2]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# make this notebook's output stable across runs
np.random.seed(0)

Question 1.1: Implement the cost function cost/objective function:
<img src="https://miro.medium.com/max/688/1*JAS6rUTO7TDlrv4XZSMbsA.png" alt="drawing" width="600"/>


In [ ]:
def compute_cost(W, X, Y,reg_strength=1000):
    N = X.dim
    C = reg_strength

    w_sq = np.linalg.norm(W)

    hinge_losses = np.maximum(0, 1 - Y * (X.dot(W)))
    hinge_loss = np.sum(hinge_losses) / N

    return 0.5 * w_sq + C * hinge_loss

Question 1.2: Write a method that calculate the cost gradient:
<img src="https://miro.medium.com/max/866/1*ww3F21VMVGp2NKhm0VTesA.png" alt="drawing" width="600"/>

In [ ]:
def calculate_cost_gradient(W, X_batch, Y_batch, reg_strength=1000):
    distance = #TO DO
    dw = #TO DO
    for #TO DO:
        if #TO DO:
            
        else:
            
    return dw

Question 1.3: Write a method that performs stochastic Gradient descent as follows:
- Caluclate the gradient of cost function i.e. ∇J(w)
- Update the weights in the opposite direction to the gradient: w = w — ∝(∇J(w))
- Repeat until conversion or until 5000 epochs are reached

In [ ]:
def sgd(data, outputs, learning_rate = 0.0001, max_epochs = 5000):
    weights = #TO DO
    nth = #TO DO
    prev_cost = #TO DO
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = #TO DO
        for ind, x in enumerate(X):#TO DO
            ascent = #TO DO
            weights = #TO DO
            # convergence check on 2^nth epoch
            if epoch == #TO DO
                cost = #TO DO
                print("Epoch is:{} and Cost is: {}".format(epoch, cost))
                # stoppage criterion
                if #TO DO:
                    return weights
                prev_cost = #TO DO
               
    return weights

# Dataset

In [4]:
data = pd.read_csv('data_banknote_authentication.csv')

Y = data.iloc[:, -1]  
X = data.iloc[:, 1:4]
X.insert(loc=len(X.columns), column='intercept', value=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

FileNotFoundError: [Errno 2] No such file or directory: 'data_banknote_authentication.csv'

Question 4: Train and evaluate an SVC using the banknote_authentication data

In [ ]:
# train the model
print("training started...")
W = sgd(X_train.to_numpy(), y_train.to_numpy())
print("training finished.")
print("weights are: {}".format(W))

# testing the model on test set
y_test_predicted = #TO DO
for i in range(X_test.shape[0]):
    yp = #TO DO #model
    y_test_predicted = #TO DO
print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), y_test_predicted)))

[Bonus] Question 5: Train and evaluate an SKLEARN SVC model, and compare the results to your model 

Question 6: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.